<strong>Algorithme du voyageur de commerce (TSP)</strong>

In [44]:
import json
from pprint import pprint
# !pip3 install geopy
from geopy.distance import distance
import numpy as np
# !pip3 install fiona
# !pip3 install gdal
# !pip3 install proj
import geopandas as gpd
import pandas as pd

In [45]:
### comptons le nombre de features de notre jeu de données
with open('../data/reanimation.geojson') as f:
    data = json.load(f)

num_features = len(data['features'])

print("Le fichier GeoJSON reanimation.geojson contient", num_features, "features.")


Le fichier GeoJSON reanimation.geojson contient 295 features.


Il va falloir créer une matrice des distances entre chaque points. on va faire cela avec un csv et avec la formule de calcul de distance avec excel

In [46]:

# Créer une liste pour stocker les noms
names = []

# Boucle à travers les "features" dans le fichier GeoJSON
for feature in data['features']:
    # Vérifier si le champ "name" est présent dans le "feature"
    if 'name' in feature['properties']:
        name = feature['properties']['name']
        # Vérifier si le nom est déjà présent dans la liste
        if name in names:
            print("Le nom", name, "est en double.")
        else:
            names.append(name)

# Afficher un message si tous les noms sont uniques
if len(names) == len(data['features']):
    print("Tous les noms sont uniques.")


Tous les noms sont uniques.


In [47]:


# Lire le fichier GeoJSON en utilisant geopandas
gdf = gpd.read_file('../data/reanimation.geojson')

# Convertir en DataFrame Pandas
# df = gdf.drop('geometry', axis=1)

# Afficher les 5 premières lignes du DataFrame
pd.options.display.max_columns = None
gdf

,id,@id,CLC:id,addr:city,addr:country,addr:full,addr:housenumber,addr:place,addr:postcode,addr:street,addr:streetnumber,alt_name,alt_name_1,amenity,architect,architect:wikipedia,barrier,building,building:levels,capacity,children,contact:city,contact:facebook,contact:fax,contact:housenumber,contact:phone,contact:postcode,contact:street,contact:twitter,contact:website,description,dispensing,electricity,email,emergency,emergency:description,facebook,fax,foot,health_amenity:type,health_facility:type,healthcare,healthcare:speciality,height,heritage,heritage:operator,internet_access,internet_access:fee,internet_access:ssid,landuse,layer,level,loc_name,mapillary,medical_system:western,mhs:inscription_date,military,name,name:br,name:de,name:en,name:fr,name:pl,note,old_name,old_name:wikipedia,opening_hours,operator,operator:type,operator:wikidata,phone,ref:FR:FANTOIR,ref:FR:FINESS,ref:FR:NAF,ref:FR:SIREN,ref:FR:SIRET,ref:mhs,roof:levels,short_name,shortest_name,source,source:FINESS,source:addr,source:addr:housenumber:ref,source:addr:version,source:heritage,source:name:br,source:ref:FR:FINESS,start_date,survey:date,toilets:wheelchair,twitter,type,type:FR:FINESS,updated,url,wall,website,wheelchair,wikidata,wikipedia,@geometry,geometry
0,relation/214949,relation/214949,NaN,Lille,NaN,NaN,44,NaN,59000,Avenue Marx Dormoy,NaN,NaN,NaN,hospital,NaN,NaN,NaN,hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+33 3 20 22 72 82,NaN,NaN,NaN,hospital,intensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DAPG3rfu50_hGnKMwcDwWg,NaN,NaN,NaN,Hôpital privé Le Bois,NaN,NaN,NaN,NaN,NaN,NaN,Polyclinique du Bois,NaN,NaN,Ramsay Santé,private,Q3123085,+33 3 59 75 75 75,NaN,590780268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cadastre-dgi-fr source : Direction Générale de...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-13,NaN,NaN,multipolygon,365,NaN,NaN,NaN,https://hopital-prive-le-bois-lille.ramsaygds.fr/,NaN,NaN,NaN,center,POINT (3.03119 50.63449)
1,relation/1221207,relation/1221207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hospital,NaN,NaN,NaN,hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,intensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,American Hospital of Paris,NaN,NaN,NaN,Hôpital américain de Paris,NaN,NaN,NaN,NaN,NaN,NaN,private,NaN,NaN,NaN,920008539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multipolygon,365,NaN,NaN,NaN,NaN,NaN,Q470085,NaN,center,POINT (2.27242 48.89254)
2,relation/1227710,relation/1227710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hospital,NaN,NaN,NaN,hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hospital,intensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hôpital Jeanne de Flandre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,public,NaN,NaN,NaN,590006607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cadastre-dgi-fr source : Direction Générale de...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multipolygon,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,center,POINT (3.03290 50.60574)
3,relation/1250218,relation/1250218,NaN,Lille,FR,NaN,NaN,NaN,59037,NaN,NaN,NaN,NaN,hospital,NaN,NaN,NaN,hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,hospital,intensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hôpital Roger Salengro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,public,NaN,NaN,NaN,590796975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cadastre-dgi-fr source : Direction Générale de...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multipolygon,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,center,POINT (3.03289 50.60751)
4,relation/1471734,relation/1471734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hôpital de Valence,NaN,hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,+33 4 75 75 72 40,NaN,NaN,NaN,NaN,intensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Centre hospitalier de Valence,NaN,NaN,NaN,NaN,NaN,NaN,

In [48]:
### Afficher les colonnes qui n'ont pas de Nan
no_nans = []
for col in gdf.columns:
    if not gdf[col].isna().any():
        no_nans.append(col)

# Afficher les colonnes qui ne contiennent pas de NaN
print("Colonnes sans NaN : ", no_nans,"\n")

###afficher si il y a des lignes qui ont le même name
print("value_counts : \n",gdf.name.value_counts())

Colonnes sans NaN :  ['id', '@id', 'amenity', 'healthcare:speciality', 'name', 'ref:FR:FINESS', 'geometry'] 

value_counts : 
 Hôpital privé Le Bois                    1
Hôpital Pierre Bérégovoy                 1
Centre hospitalier de Roubaix            1
Centre hospitalier de Tourcoing          1
Hôpital Saint-Joseph                     1
                                        ..
Hôpital privé de Parly 2 Le Chesnay      1
Centre hospitalier de Mâcon              1
Centre hospitalier de Lons-le-Saunier    1
Centre hospitalier de Troyes             1
Hôpital Gui de Chauliac                  1
Name: name, Length: 295, dtype: int64


<strong>Résolution du TSP</strong>

Dans la pertie clustering, Paul a fait des clusters et calculé les coordonnées des centroïdes.

Ce qu'on va faire maintenant c'est calculer les tournées.

On va dans un premier temps calculer les distances à vol d'oiseau entre les points.

On va ensuite créer les tournées pour chaque cluster.

**Création de la matrice avec les distances à vol d'oiseau**

On récupère le csv qui contient les leiux et les coordonnées, ainsi que le numéro du cluster correspondant.

In [49]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from geopy.distance import great_circle
df = pd.read_csv('../Clustering/points+centroides.csv')
# Sélection des colonnes à utiliser pour calculer les distances
cols = ['name', 'latitude', 'longitude', 'cluster']
df_sub = df[cols]

# Extraction des noms de points
points = df_sub['name']

# Calcul de la matrice de distances
coords = df_sub[['latitude', 'longitude']].values
dist_matrix = squareform(pdist(coords, (lambda u,v: great_circle(u,v).km)))

# Création d'un dataframe à partir de la matrice de distances
dist_df = pd.DataFrame(dist_matrix, index=points, columns=points)

# Ajout des colonnes 'latitude', 'longitude' et 'cluster' à la matrice de distances
dist_df = df_sub[['name', 'latitude', 'longitude', 'cluster']].merge(dist_df, left_on='name', right_index=True)

# Affichage et exportation en CSV de la matrice de distances
dist_df.set_index('name', inplace=True)
dist_df.to_csv('distanceMatrice.csv', index=True)
dist_df

,latitude,longitude,cluster,Hôpital privé Le Bois,American Hospital of Paris,Hôpital Jeanne de Flandre,Hôpital Roger Salengro,Centre hospitalier de Valence,Hôpital Henri Mondor,Institut Calot,Hôpital de la Mère et de l’Enfant,Hôpital européen,Centre Henri Becquerel,Institut Universitaire du Cancer de Toulouse - Oncopole,Centre hospitalier Annecy-Genevois,Centre Hospitalier Saint-Joseph Saint-Luc,Hôpital Louis Mourier,Hôpital Foch,CHU Nord de Grenoble,Hôpital de la Cavale Blanche,Institut Coeur Poumon,Centre hospitalier de Niort,Hôpital Privé d'Antony,Clinique Mathilde,Centre Hospitalier de Narbonne,Hôpital Tenon,GHU Paris Psychiatrie & neurosciences - site Sainte-Anne,Fondation ophtalmologique Adolphe de Rothschild,Hôpital Cochin,Institut Mutualiste Montsouris,Hôpital Européen Georges Pompidou,Hôpital Armand-Trousseau,Hôpital Lariboisière,Centre Hospitalier de Roanne,Hôpital du Scorff,Centre hospitalier intercommunal de Créteil,Hôpital Purpan,Groupe hospitalier Pellegrin,Hôpital d'Instruction des Armées Robert Picqué,Centre hospitalier de Fontainebleau,Clinique du Millénaire,Centre hospitalier de Cholet,Hôpital Saint-Louis,Centre hospitalier de Perpignan,Hôpital Haut-Lévêque,Centre Chirurgical Marie Lannelongue,Centre Hospitalier Intercommunal Lucie et Raymond Aubrac,Hôpital Gabriel Montpied,Centre hospitalier de Meulan-en-Yvelines,Hôpital Sud,Hôpital Bretonneau,Hôpital Paul Brousse,Hôpital Robert Debré,Hôpital Bicêtre,Hôpital d'instruction des armées Sainte-Anne,Hôpital Avicenne,Centre hospitalier intercommunal Fréjus Saint-Raphaël,Centre hospitalier de Dieppe,Clinique d'Occitanie,Centre hospitalier de Longjumeau,Clinique de l'Union,Hôpital Émile Muller,Centre hospitalier de Pau,Centre hospitalier d'Albi,Hôpital de la Conception,Hôpital Simone Veil,Centre hospitalier intercommunal André Grégoire,Hôpital Jean Verdier,Hopital Beaujon,Centre hospitalier Comminges Pyrénées,Hôpital d'Instruction des Armées Bégin,Centre hospitalier,Hôpital Bichat-Claude Bernard,Centre hospitalier de Béziers,Centre hospitalier intercommunal de Cornouaille,Centre hospitalier intercommunal de Poissy,Hôpital Delafontaine,Centre hospitalier Jacques Lacarin,Hôpital Civil,Hôpital privé les Franciscaines,Centre hospitalier de Sambre-Avesnois,Centre Hospitalier Nord Ouest Villefranche,Hôpital de Hautepierre,Centre Hospitalier Régional Universitaire de Lille,Centre hospitalier de Cambrai,Hôpital Trousseau,Hôpital Central,Hôpital Charles Nicolle,Centre hospitalier intercommunal Robert Ballanger,Centre hospitalier de Mont-de-Marsan - Site de Sainte-Anne,Hôpial privé du Vert-Galant,Centre Hospitalier Henri Mondor,Centre hospitalier François Quesnay,Hôpital Ambroise Paré,Centre hospitalier des Pays de Morlaix,Hôpital Pasteur,Centre hospitalier de Haguenau,Centre hospitalier de Mont-de-Marsan,Centre hospitalier de Troyes,Centre hospitalier de Lons-le-Saunier,Centre hospitalier de Mâcon,Hôpital privé de Parly 2 Le Chesnay,Centre hospitalier de Bigorre - La Gespe,Polyclinique Bordeaux Nord Aquitaine,Centre hospitalier Ardèche Nord,Centre hospitalier de la Côte Basque - Site Cam de Prats,Centre hospitalier d’Auxerre,Centre hospitalier du Pays d'Aix,Centre hospitalier de Cahors,Clinique Mutualiste Chirurgicale,Hôpital Privé de la Loire,Hôpital Lozère,Centre hospitalier de Guéret,Hôpital Jacques Monod,Centre hospitalier d'Aubagne,Centre hospitalier universitaire de Limoges,Hôpital privé Jacques Cartier,Centre Hospitalier Sainte-Catherine,Clinique des Ormeaux,Centre hospitalier de Périgueux,Centre hospitalier de Versailles,Centre hospitalier de Creil,Centre hospitalier des vallées de l’Ariège - Site de Foix,Centre hospitalier de Moulins-Yzeure,Centre hospitalier de Lens,Hôpital Croix Saint-Simon,Hôpital privé La Casamance,Centre hospitalier Geneviève de Gaulle Anthonioz,Grand Hôpital de l'Est Francilien - Site de Marne-la-Vallée,Centre hospitalier de Brive-la-Gaillarde,Hôpital Saint-Philibert,Centre hospitalier de Meaux,Clinique du Pont de Chaume,Hôpital d'instruction 

**On a une matrice des distances, on va pouvoir maintenant créer les tournées pour chaque cluster**

In [50]:
dist_df = dist_df

# on va créer les tournées pour chaque cluster
centroide = pd.read_csv("../Clustering/centroids.csv").name # on crée un dataframe qui ne contient que les  centroides car ce sont els points de départ de chaque tournée

clusters = dist_df.cluster.unique()
def dictPoints(clusters = clusters):
    """crée un dictionnaire. chaque entrée est le numéro de cluster. chque entrée contient une liste des points qui sont dans le cluster.

    Returns:
        dict: clef = id cluster. entrée : liste des points de ce cluster
    """
    dicTournee = {}
    # Création d'une liste vide pour chaque cluster
    for elt in clusters:
        dicTournee[elt] = []
    
    #on ajoute chaque point dans son entrée de dictionnaire
    for idx, row in df.iterrows():
        cluster = row['cluster']
        name = row['name']
        if cluster in dicTournee:
            if name[0:9] != 'Centroïde':
                dicTournee[cluster].append(name)
    return dicTournee
    

def distance(point1, point2,df=dist_df):
    """
    point1 et point2 sont des str.\n
    renvoie la distance dans la matrice de distance entre le point1 et le point2.
    """
    distance = df[point1][point2]
    distance = int(round(float(distance)))
    return distance





def plus_proche_voisin(point1, points_restants):
    """
    Calcule le point le plus proche de point1, en augmentant la valeur minimale de la distance 
    tant que le point le plus proche et point1 n'ont pas le même cluster

    Args:
        point1 (str): Nom du point pour lequel on cherche le plus proche voisin
        df (DataFrame): DataFrame des distances
        points_restants (list): Liste des points restants à parcourir

    Returns:
        str: Nom du point le plus proche appartenant au même cluster que point1
    """
    res = None
    dist=9999 # on met une grande distance qu'on cherche à minimiser au fur et à mesure
    for elt in points_restants:
        if dist > distance(elt,point1):
            dist = distance(elt,point1)
            res = elt
    return res



def IdToCentroide(id, df=dist_df):
    """fonction qui prends un entier  le numéro de cluster et renvoie le centroide en str qui correspond

    Args:
        id (int): id du cluster correspondant au centroide
    
    Return (str) : le centoride correspondant au cluster
    """
    for index in df.index:
        if index == "Centroïde_"+str(id): # cherche l'index correspondant à l'id demandé
            return index


def tournees(idCentroide, seuil = 200, df = dist_df):
    """calcule les tournées pour le cluster donnés et les points qu'il doit parcourir.


    Args:
        centroide (int): le centroide correspondant au cluster et aux points.
        seuil (int) : nombre de km max que la gars peut faire en une tournée. 

    Returns : res (lis) : une liste de liste qui contient les tournées pour ce cluster
    """
    nb=0
    res=[]
    nameCentroide = IdToCentroide(idCentroide)
    liste_points = dictPoints(clusters = clusters)[idCentroide] # on récupère les ppoints que doit parcourir depuis ce centroide.
    liste_points_restants = liste_points.copy()
    while len(liste_points_restants) > 0 :
        tournee = [nameCentroide] # on initialise la tournée avec comme point de départ le centroide.
        distance_tournee = 0

        #on va chercher le point le plus proche
        proche = plus_proche_voisin(tournee[-1], points_restants = liste_points_restants) #on cherche le point le plus proche du avant dernier point de la tournée
        tournee.append(proche)
        liste_points_restants.remove(proche)
        distance_tournee += distance(point1=tournee[0], point2=tournee[-1])
        # on va rajouter le point plus proche seulement si on ne dépasse pas le seuil de km par jour en comptant le retour au point d'arrivée.
        if len(liste_points_restants) == 0:
            tournee.append(nameCentroide)
            res.append(tournee)
        while len(liste_points_restants) > 0:
            proche = plus_proche_voisin(point1= tournee[-1],  points_restants = liste_points_restants) #on cherche le point le plus proche du avant dernier point de la tournée
            if (distance_tournee + distance(tournee[-1], proche)+distance(proche, nameCentroide)) < seuil:
                #on va ajouter le proche en avant dernière position de la tournée.
                distance_tournee+= distance(point1 = tournee[-1], point2 = proche) # ajoute à la distance de la tournée le nombre de km supplémentaire
                tournee.append(proche) #ajouter le point

                liste_points_restants.remove(proche) #supprimer le point qu'on vient d'ajouter de la liste des villes restantes. 
                if len(liste_points_restants) == 0:
                    tournee.append(nameCentroide)
                    res.append(tournee)

            else :
                tournee.append(nameCentroide)
                res.append(tournee)
                break
        
    return res


def algoTSP():
    """calculer les tournées cluster par cluster
    """
    dict = {}
    for cluster in clusters:
        var = tournees(cluster, seuil = 800, df = dist_df)
        dict[cluster] = var
    return dict


res = algoTSP()
print(res)
for entree in res.values():
    for sous_liste in entree:
        print(sous_liste)




{0: [['Centroïde_0', 'Hôpital Simone Veil', "Centre hospitalier d'Argenteuil", 'Hôpital Louis Mourier', 'American Hospital of Paris', 'Hopital Beaujon', 'Hôpital Bichat-Claude Bernard', 'Hôpital Lariboisière', 'Hôpital Saint-Louis', 'Fondation ophtalmologique Adolphe de Rothschild', 'Hôpital Tenon', 'Hôpital Croix Saint-Simon', 'Hôpital Armand-Trousseau', "Hôpital d'Instruction des Armées Bégin", 'Hôpital Saint-Antoine', 'Hôpital Pitié-Salpêtrière', 'Clinique Geoffroy Saint-Hilaire', 'Hôpital Cochin', 'GHU Paris Psychiatrie & neurosciences - site Sainte-Anne', 'Institut Mutualiste Montsouris', 'Hôpital Bicêtre', 'Hôpital Paul Brousse', 'Institut Gustave Roussy', "Hôpital Privé d'Antony", 'Hôpital privé Jacques Cartier', 'Centre hospitalier de Longjumeau', "Centre hospitalier d'Orsay", 'Centre Chirurgical Marie Lannelongue', 'Hôpital Antoine Béclère', "Hôpital d'Instruction des Armées Percy", 'Hôpital Européen Georges Pompidou', 'Hôpital Ambroise Paré', "Clinique du Val d'Or", 'Hôpital 

On va afficher les distances de chaque tournées

In [51]:
# res = algoTSP()
liste_de_tournees=[]
for entree in res.values():
    for sous_liste in entree:
        liste_de_tournees.append(sous_liste)

###calcule des distances:
distances_tournees=[]
for tournees in liste_de_tournees:
    dist=0
    for i in range(1,len(tournees)):
        dist+= distance(tournees[i-1], tournees[i])
    distances_tournees.append(dist)

###nombre de points dans la tournées
nombre_points=[]
total=0
for tournees in liste_de_tournees:
    nb_points_de_la_tournee=0
    for point in tournees:
        nb_points_de_la_tournee+=1
    nb_points_de_la_tournee-=2 #on enlève les points de départs et arrivés
    total += nb_points_de_la_tournee
    nombre_points.append(nb_points_de_la_tournee)


###Export des tournees et distances et nombre de points dans un csv
df_tournees = pd.DataFrame(liste_de_tournees)
df_tournees.insert(0, 'distance', distances_tournees)
df_tournees.insert(1, 'nombre_points', nombre_points)
df_tournees.to_csv("tournees.csv")

### affichage des distances total, moyenne, nombre de points parcourus au total, en moyenne.
print("La distance total parcourue est ",df_tournees["distance"].sum(), " km.")
print("La distance moyenne par tournée est ",round(df_tournees["distance"].mean()),"km")
print("On a parcouru ",round(df_tournees["nombre_points"].sum()),"points.")
print("On parcoure en moyenne ",round(df_tournees["nombre_points"].mean()), "points par tournée")

df_tournees


La distance total parcourue est  12344  km.
La distance moyenne par tournée est  686 km
On a parcouru  295 points.
On parcoure en moyenne  16 points par tournée


,distance,nombre_points,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73
0,791,72,Centroïde_0,Hôpital Simone Veil,Centre hospitalier d'Argenteuil,Hôpital Louis Mourier,American Hospital of Paris,Hopital Beaujon,Hôpital Bichat-Claude Bernard,Hôpital Lariboisière,Hôpital Saint-Louis,Fondation ophtalmologique Adolphe de Rothschild,Hôpital Tenon,Hôpital Croix Saint-Simon,Hôpital Armand-Trousseau,Hôpital d'Instruction des Armées Bégin,Hôpital Saint-Antoine,Hôpital Pitié-Salpêtrière,Clinique Geoffroy Saint-Hilaire,Hôpital Cochin,GHU Paris Psychiatrie & neurosciences - site S...,Institut Mutualiste Montsouris,Hôpital Bicêtre,Hôpital Paul Brousse,Institut Gustave Roussy,Hôpital Privé d'Antony,Hôpital privé Jacques Cartier,Centre hospitalier de Longjumeau,Centre hospitalier d'Orsay,Centre Chirurgical Marie Lannelongue,Hôpital Antoine Béclère,Hôpital d'Instruction des Armées Percy,Hôpital Européen Georges Pompidou,Hôpital Ambroise Paré,Clinique du Val d'Or,Hôpital Foch,Hôpital Raymond Poincaré,Centre hospitalier de Versailles,Hôpital privé de Parly 2 Le Chesnay,Centre Médico-Chirurgical de l'Europe,Centre hospitalier intercommunal de Poissy,Centre hospitalier de Meulan-en-Yvelines,Centre hospitalier René Dubos,Centre Cardiologique du Nord,Hôpital Delafontaine,Hôpital Européen de Paris,Hôpital Robert Debré,Hôpital Avicenne,Centre hospitalier intercommunal André Grégoire,Hôpital Jean Verdier,Centre hospitalier intercommunal Robert Ballanger,Hôpial privé du Vert-Galant,Groupe hospitalier intercommunal Le Raincy-Mon...,Hôpital privé de Marne-Chantereine,Hôpital Saint-Camille,Centre hospitalier intercommunal de Créteil,Hôpital Henri Mondor,Centre Hospitalier Intercommunal Lucie et Raym...,Hôpital privé du Val d'Yerres,Hôpital privé Claude Galien,Centre Hospitalier Sud Francilien,Centre hospitalier de Melun,Centre hospitalier de Fontainebleau,Centre hospitalier d'Étampes,Centre Hospitalier de Rambouillet,Hôpital privé de l'Ouest Parisien,Hôpital Necker Enfants Malades,Centre Hospitalier de Gonesse,Centre hospitalier de Creil,Centre hospitalier de Compiègne,Centre hospitalier de Soissons,Centre hospitalier de Laon,Centre hospitalier de Saint-Quentin,Centre hospitalier de Cambrai,Centre hospitalier de Douai,Centroïde_0
1,797,28,Centroïde_0,Centre hospitalier de Beauvais,CHU Amiens Sud,Centre hospitalier d'Abbeville,Centre Hospitalier de l'Arrondissement de Mont...,Institut Calot,Centre hospitalier de Boulogne-sur-Mer,Centre hospitalier de Calais,Centre hospitalier de Dunkerque,Centre hospitalier de la Région de Saint-Omer,Centre hospitalier de Béthune,Centre hospitalier de Lens,Centre hospitalier d'Arras,Hôpital Jeanne de Flandre,Hôpital Roger Salengro,Institut Coeur Poumon,Centre Hospitalier Régional Universitaire de L...,Hôpital Claude Huriez,Hôpital privé Le Bois,Hôpital Saint-Philibert,Centre hospitalier d'Armentières,Centre hospitalier de Roubaix,Centre hospitalier de Tourcoing,Centre Hospitalier de Valenciennes,Centre hospitalier de Sambre-Avesnois,CHU de Reims,Polyclinique de Reims Bezannes,Centre hospitalier de Château-Thierry,Centre hospitalier de Meaux,Centroïde_0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,792,19,Centroïde_0,Centre hospitalier François Quesnay,Centre hospitalier de Dreux,Centre Hospitalier Louis Pasteur,Centre hospitalier Eure-Seine,Centre hospitalier d'Elbeuf,Clinique Mathilde,Centre Henri Becquerel,Hôpital Charles Nicolle,Clinique Saint-Hilaire,Clinique de l'Europe,Clinique du Cèdre,Clinique Tous Vents,Hôpital Jacques Monod,Clinique des Ormeaux,Centre hospitalier Robert Bisson,Centre hospitalier d'Alençon,Nouvelle Clinique de Tours Plus,Hôpital Bretonneau,Hôpital Clocheville,Centr